### Using Urth Widgets in Scala

In [ ]:
%%html
<link rel="stylesheet" type="text/css" href="bcard.css">

Add, import and initialize the widget system:

In [ ]:
// modify to IP and Port of this notebook server
%addjar http://localhost:8888/nbextensions/urth_widgets/urth-widgets.jar

In [ ]:
import urth.widgets._
initWidgets

Import Polymer elements:

In [ ]:
%%html
<link rel='import' href='urth_components/paper-slider/paper-slider.html' 
        is='urth-core-import' package='PolymerElements/paper-slider'>

### Function widget

#### Def

In [ ]:
def math(x: Int, y: Double = 50): Double = x * y

In [ ]:
%%html
<template is="dom-bind">
<urth-core-function ref="math" arg-x="{{x}}" arg-y="{{y}}" result="{{res}}" auto></urth-core-function>
    <label>x:</label><paper-slider min="10" max="100" step="1" value="{{x}}"></paper-slider><span>{{x}}</span><br>
    <label>y:</label><paper-slider min="1" max="100" step="1" value="{{y}}"></paper-slider><span>{{y}}</span><br>
Result: <span>{{res}}</span>
</template>

#### Val

In [ ]:
val fun = (x: Int,  y: Double) => x * y

In [ ]:
%%html
<template is="dom-bind">
<urth-core-function id="test" ref="fun" arg-x="{{x}}" arg-y="{{y}}" result='{{res}}' auto></urth-core-function>
    <label>x:</label><paper-slider min="10" max="100" step="1" value="{{x}}"></paper-slider><span>{{x}}</span><br>
    <label>y:</label><paper-slider min="1" max="10" step="0.1" value="{{y}}"></paper-slider><span>{{y}}</span><br>
    Result: <span>{{res}}</span>
</template>

Note that the function below will not get automatically invoked after the ```<urth-core-function>``` is created since the arguments are not set:

In [ ]:
def moreMath(x: Int, y: Double) = x * y

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-function ref="moreMath" args="{{args}}" result="{{res}}" auto></urth-core-function>
    x: <input type="text" value="{{args.x::change}}"></input>
    y: <input type="text" value="{{args.y::change}}"></input>
    Result: <span>{{res}}</span>
</template>

#### DataFrame return value

In [ ]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._
import org.apache.spark.sql.DataFrame

def dfFunc(x: Int): DataFrame = {
    val seq = (0 until x).map(i => (i, i)).toSeq
    sqlContext.createDataFrame(seq)
}

In [ ]:
%%html
<template is="dom-bind">
<urth-core-function ref="dfFunc" arg-x="{{x}}" result='{{res}}' limit="10" auto></urth-core-function>
    <label>x:</label><paper-slider min="0" max="20" step="1" value="{{x}}"></paper-slider><span>{{x}}</span><br>
    <label>columns:</label><span>{{res.columns}}</span> <br/>
    <label>index:</label><span>{{res.index}}</span> <br/>
    <label>data:</label><span>{{res.data}}</span>
</template>

### DataFrame widget

In [ ]:
val seq = (0 until 10000).map(i => (i, i)).toSeq
val largeDataFrame = sqlContext.createDataFrame(seq)

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe ref="largeDataFrame" limit="10" value="{{x}}"></urth-core-dataframe>
    <p>Columns: <span>{{x.columns}}</span></p>
    <p>Index: <span>{{x.index}}</span></p>
    <p>Data:</p>
    <template is="dom-repeat" items="{{x.data}}">
        <span>{{item}}</span><br/>
    </template>
</template>

In [ ]:
case class Contact(firstname: String, lastname: String, title: String, email: String, phone: String, web: String)

In [ ]:
val contacts = sqlContext.createDataFrame(Seq(
    Contact("Jane", "Doe","Scala Developer", "jane@ibm.com", "123-432-5431", "http://www.ibm.com/jane"), 
    Contact("John", "Doe","Spark Engineer", "john@ibm.com", "143-421-5411", "http://www.ibm.com/john"),
    Contact("Joe", "Smith","Product Manager", "joe@ibm.com", "123-732-8421", "http://www.ibm.com/joe")))

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe ref="contacts" value="{{x}}" auto></urth-core-dataframe>
    
    <template is="dom-repeat" items="{{x.data}}">
      <div class="bcard">
        <div class="info">
            <div class="line full-name"><span>{{item.0}}</span> <span>{{item.1}}</span></div>
            <span class="line title">{{item.2}}</span>
            <span class="line phone-number">{{item.3}}</span>
            <span class="line email">{{item.4}}</span>
            <span class="line website">{{item.5}}</span>
        </div>
        <div class="logo"></div>
      </div>
    </template>
</template>

### Channels API

Interact with `urth-core-bind` `channel` variables using the following API:

#### Set Item

In [ ]:
%%html
<template is='urth-core-bind' channel='a'>
    <div>Hello from <span>{{user}}</span></div>
    Name: <input value='{{user::input}}'></input>
</template>

In [ ]:
import urth.widgets.WidgetChannels.channel

In [ ]:
channel("a").set("user", "bea")

#### Watch Item

In [ ]:
%%html
<template is='urth-core-bind' channel='b'>
    <p><span>{{x}}</span></p>
    <span>{{y}}</span>
</template>

In [ ]:
channel("b").set("x", "bean")

Now we set up a watch handler for variable `x` on channel `b`. 

Note that the first argument is of type `Option`, since an `oldVal` may not be present.

In [ ]:
val handler = (oldVal: Option[String], newVal: String) => 
    channel("b").set("y", s"Hello from the kernel! old: ${oldVal.getOrElse("")}, new: $newVal")

In [ ]:
channel("b").watch("x", handler)

In [ ]:
channel("b").set("x", "dean")

Handlers can be typed. `Seq` corresponds to JavaScript arrays and `Map` corresponds to JavaScript object.

This example shows a handler that uses `Seq`:

In [ ]:
%%html
<template is='urth-core-bind' channel='c'>
    <p><span>{{x}}</span></p>
    <span>{{y}}</span>
</template>

In [ ]:
channel("c").set("x", List(0))

In [ ]:
val handler = (oldVal: Option[Seq[Int]], newVal: Seq[Int]) => 
    channel("c").set("y", s"Hello from the kernel! old: ${oldVal.getOrElse(None)}, new: $newVal")

In [ ]:
channel("c").watch("x", handler)

In [ ]:
channel("c").set("x", List(0, 1, 2))